# Bert Tokenizer
This model pushed to huggingface [link text](https://huggingface.co/mansoorhamidzadeh/bert_classification)




In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="mansoorhamidzadeh/bert_classification")
pipe('i love you')

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.9967560172080994}]

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data='/content/drive/MyDrive/nlp/ecommerceDataset.csv'
dataset=pd.read_csv(data)
dataset['Household'].unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [ ]:
dataset['Household'].isnull().sum()
dataset.drop_duplicates(inplace=True)
dataset.dropna(inplace=True)

In [ ]:
dataset.rename(columns={'Household':"label"},inplace=True)
dataset.rename(columns={'Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal bliss.so bring home this elegant print that is lushed with rich colors that makes it nothing but sheer elegance to be to your friends and family.it would be treasured forever by whoever your lucky recipient is. Liven up your place with these intriguing paintings that are high definition hd graphic digital prints for home, office or any room.':"text"},inplace=True)

In [ ]:
dataset['label'].unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokenizer('hi')

{'input_ids': [101, 7632, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
label_encoder=LabelEncoder()
dataset['label']=label_encoder.fit_transform(dataset['label'])

In [ ]:
train_text, val_text, train_label, val_label = train_test_split(dataset['text'].tolist(), dataset['label'].tolist(), test_size=0.3)


In [ ]:
train_dataset = TextDataset(train_text, train_label, tokenizer, max_length=128)
val_dataset = TextDataset(val_text, val_label, tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:
next(iter(train_loader))

{'input_ids': tensor([[  101,  2712, 18477, 28168,  1024,  2019,  2137,  5722,  3319,  1523,
          17160,  1012,  1012,  1012, 14954,  1012,  1012,  1012,  1037,  2034,
           1011,  3446,  3538,  1997, 20957,  1010,  2975,  2149,  2025,  2069,
           2007,  1037, 14954,  6533,  1997,  1037,  3586,  2021,  1037, 17160,
          14704,  1997,  2137,  2381,  2004,  2092,  1012,  1524,  1517,  1996,
           2047,  2259,  2335,  1523, 25540, 25725,  2075,  1012,  1012,  1012,
           3435,  1011,  3048,  1012,  1012,  1012,  2062,  2084,  2074,  1037,
           3586,  1521,  1055,  6925,  1010,  2138,  1996,  4286,  2040,  3079,
           1010,  4738,  1010,  1998,  8469,  2712, 18477, 28168,  2024,  8053,
          17160,  1012,  1012,  1012,  1012,  1031,  2940,  2368, 23544,  1033,
           3065,  2019,  9313,  5848,  2005,  7851,  1037,  3586,  2679,  2061,
          14954,  2135,  2008,  1996,  8068,  5683,  2066,  1996,  7945,  1012,
           1524,  1517,  29

In [ ]:
def train_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    cor_pred = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            cor_pred += (predictions == labels).sum().item()

    return total_loss / len(dataloader), cor_pred / len(dataloader.dataset)

In [ ]:
epochs = 3
for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, optimizer)
    val_loss, val_accuracy = evaluate(model, val_loader)
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Training Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

print("Training complete.")


Evaluating: 100%|██████████| 4171/4171 [01:27<00:00, 47.68it/s]


Epoch 1/3
Training Loss: 0.2347
Validation Loss: 0.1723
Validation Accuracy: 0.9538


Evaluating: 100%|██████████| 4171/4171 [01:26<00:00, 48.47it/s]


Epoch 2/3
Training Loss: 0.1445
Validation Loss: 0.1589
Validation Accuracy: 0.9608


Evaluating: 100%|██████████| 4171/4171 [01:28<00:00, 47.18it/s]

Epoch 3/3
Training Loss: 0.0995
Validation Loss: 0.2213
Validation Accuracy: 0.9392
Training complete.


In [ ]:
model.save_pretrained('bert_classification')
tokenizer.save_pretrained('bert_classification')


('bert_classification/tokenizer_config.json',
 'bert_classification/special_tokens_map.json',
 'bert_classification/vocab.txt',
 'bert_classification/added_tokens.json')

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert_classification')
tokenizer = BertTokenizer.from_pretrained('bert_classification')
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def predict(text, model, tokenizer, max_length=128):
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).item()
    return predictions


In [ ]:
labels=['Household', 'Books', 'Clothing & Accessories', 'Electronics']


In [ ]:
def main(dataset):
  text=dataset['text']
  label=dataset['label']
  predicted= predict(text,model,tokenizer)
  print(f'Correct answer is "{labels[label]}" --> predicted : "{labels[predicted]}"')


In [ ]:
main(dataset.iloc[10])


In [ ]:

shutil.make_archive('bert_classification', 'zip', 'bert_classification')

'/content/bert_classification.zip'

In [ ]:
!cp bert_classification.zip /content/drive/MyDrive/nlp/

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
from huggingface_hub import notebook_login, HfApi, HfFolder

In [ ]:
notebook_login()

In [ ]:
api = HfApi()

In [ ]:
user_info = api.whoami()
username = user_info['name']
print(username)

In [ ]:
repo_id = f"{username}/bert_classification"
repo_url = api.create_repo(repo_id, exist_ok=True)

In [ ]:
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mansoorhamidzadeh/bert_classification/commit/ff5b932a5bceaaf9b2216cff726121f6633ee427', commit_message='Upload tokenizer', commit_description='', oid='ff5b932a5bceaaf9b2216cff726121f6633ee427', pr_url=None, pr_revision=None, pr_num=None)